<a href="https://colab.research.google.com/github/KickDirtBaseball/register/blob/master/Framing_Model_0_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pybaseball scikit-learn joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 37.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from pybaseball import statcast
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
import joblib

In [ ]:
from datetime import datetime

# Pull data from the start of the 2025 season to today
df = statcast(start_dt='2025-03-28', end_dt=datetime.now().strftime('%Y-%m-%d'))

This is a large query, it may take a moment to complete


/usr/local/lib/python3.11/dist-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
  0%|          | 0/77 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_cop

In [ ]:
df_model = df[df['description'].isin(['called_strike', 'ball'])].copy()

In [ ]:
required_cols = [
    'pitch_type', 'stand', 'p_throws', 'balls', 'strikes',
    'zone', 'release_speed', 'release_pos_x', 'release_pos_z',
    'plate_x', 'plate_z', 'sz_top', 'sz_bot', 'release_extension'
]

df_model = df_model.dropna(subset=required_cols)
print("Valid rows after filtering:", len(df_model))

Valid rows after filtering: 146202


In [ ]:
df_model['called_strike'] = (df_model['description'] == 'called_strike').astype(int)
df_model['count'] = df_model['balls'].astype(str) + '-' + df_model['strikes'].astype(str)
df_model['hand_matchup'] = df_model['stand'] + 'vs' + df_model['p_throws']

In [ ]:
features = [
    'pitch_type', 'stand', 'p_throws', 'count', 'hand_matchup',
    'zone', 'release_speed', 'release_pos_x', 'release_pos_z',
    'plate_x', 'plate_z', 'sz_top', 'sz_bot', 'release_extension'
]
target = 'called_strike'

X = df_model[features]
y = df_model[target]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Column types
num_cols = ['zone', 'release_speed', 'release_pos_x', 'release_pos_z',
            'plate_x', 'plate_z', 'sz_top', 'sz_bot', 'release_extension']
cat_cols = ['pitch_type', 'stand', 'p_throws', 'count', 'hand_matchup']

# Preprocessing
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

In [ ]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['zone', 'release_speed',
                                                   'release_pos_x',
                                                   'release_pos_z', 'plate_x',
                                                   'plate_z', 'sz_top',
                                                   'sz_bot',
                                                   'release_extension']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['pitch_type', 'stand',
                                                   'p_throws', 'count',
                                                   'hand_matchup'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score

y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))

Accuracy: 0.9343109627643567
ROC AUC: 0.9818975534524075


In [ ]:
import joblib

joblib.dump(pipeline.named_steps['classifier'], 'framing_model_2025.joblib')
joblib.dump(pipeline.named_steps['preprocessor'], 'framing_model_preprocessor_2025.joblib')

['framing_model_preprocessor_2025.joblib']

In [ ]:
from google.colab import files

# Download both saved files
files.download("framing_model_2025.joblib")
files.download("framing_model_preprocessor_2025.joblib")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>